Copyright 2023 Borislav Rumenov Varbanov

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Simple 3D Cube Rendering from Side Perspective
This notebook demonstrates a minimal 3D pipeline that rotates and projects
a cube before rasterizing it with Matplotlib.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Polygon

x_res, y_res = 800, 640

cube_vertices = np.array([
    [-1,-1,-1],
    [ 1,-1,-1],
    [ 1, 1,-1],
    [-1, 1,-1],
    [-1,-1, 1],
    [ 1,-1, 1],
    [ 1, 1, 1],
    [-1, 1, 1]
])

faces = [
    [0,1,2,3],
    [4,5,6,7],
    [0,1,5,4],
    [2,3,7,6],
    [1,2,6,5],
    [0,3,7,4]
]

fig, ax = plt.subplots()
ax.set_xlim(0, x_res)
ax.set_ylim(0, y_res)
ax.set_aspect('equal')
ax.axis('off')

polys = [Polygon(np.zeros((4,2)), edgecolor='black', alpha=0.5) for _ in faces]
for p in polys:
    ax.add_patch(p)

def project(v, d=5):
    z = v[:,2] + d
    return np.column_stack((v[:,0]/z, v[:,1]/z))

def update(frame):
    ang = frame * 0.02
    R_y = np.array([
        [np.cos(ang),0,np.sin(ang)],
        [0,1,0],
        [-np.sin(ang),0,np.cos(ang)]
    ])
    R_x = np.array([
        [1,0,0],
        [0,np.cos(ang),-np.sin(ang)],
        [0,np.sin(ang),np.cos(ang)]
    ])
    rotated = cube_vertices @ R_y.T @ R_x.T
    proj = project(rotated)
    proj[:,0] = (proj[:,0] + 1) * x_res/2
    proj[:,1] = (1 - proj[:,1]) * y_res/2
    for poly, face in zip(polys, faces):
        poly.set_xy(proj[face])
    return polys

ani = FuncAnimation(fig, update, frames=360, interval=1000/60, blit=True)
plt.show()

